In [2]:
#pip install pandahouse
import pandahouse as ph
import pandas as pd

In [3]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
'database':'default',
'user':'student',
'password':'dpo_python_2020'
}

In [4]:
connection_test = dict(database='test',
host='http://clickhouse.beslan.pro:8080',
user='student-rw',
password='656e2b0c9c')

In [14]:
df_in = pd.read_csv('/home/jupyter-l.fedoseeva-12/SQL/medium.csv', sep='\t')

In [15]:
df_in

,published,domain,url
0,2012-08-13 22:54:53.510Z,medium.com,https://medium.com/policy/medium-terms-of-serv...
1,2012-08-13 22:57:17.248Z,medium.com,https://medium.com/policy/medium-privacy-polic...
2,2016-11-04 23:40:43.364Z,medium.com,https://medium.com/@Medium/personalize-your-me...
3,2016-12-24 18:21:13.427Z,medium.com,https://medium.com/holiday-poems/xmas-morning-...
4,2015-09-22 21:37:48.207Z,blog.medium.com,https://blog.medium.com/taking-a-side-on-net-n...
...,...,...,...
92186,2017-05-23 17:15:09.193Z,medium.com,https://medium.com/@KyleAndrews1994/its-time-t...
92187,2017-01-31 09:59:12.316Z,medium.com,https://medium.com/silent-protagonist/next-wee...
92188,2017-02-16 14:15:20.116Z,medium.com,https://medium.com/@utapartment26/there-used-t...
92189,2017-06-27 10:16:21.668Z,medium.com,https://medium.com/silent-protagonist/the-snes...


Пример 1: выводим первые 10 UserID, записываем в датафрейм q_test

In [16]:
q = '''
SELECT
UserID
FROM
{db}.checks
LIMIT 10
'''
# отправляем запрос и записываем результат в пандасовский датафрейм
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,UserID
0,18446583642950580515
1,18446535622689003675
2,18446130411954852964
3,18446003252714243011
4,18445948434655311802
5,18445927732647659917
6,18445761122620052505
7,18445655133428855896
8,18445587876544434519
9,18445586451093345117


Пример 2: создаем таблицу с помощью CREATE TABLE

In [29]:
q = """
CREATE TABLE test.example_table_l (
id UInt32,
supplementary_id UInt32,
comments String
)
ENGINE = MergeTree
ORDER BY (id, supplementary_id)
"""
ph.execute(query=q, connection=connection_test)

b''

Пример 3 записи данных в ClickHouse в уже созданную таблицу medium :

In [12]:
df = []

In [31]:
# записываем данные из pandas в clickhouse
ph.to_clickhouse(df_in, 'medium', index=False, connection=connection_test)

92191

In [28]:
# проверяем, что записалось
query_test = """
SELECT *
FROM test.medium
LIMIT 100
"""
df_new = ph.read_clickhouse(query_test, connection=connection_test)
df_new

,published,domain,url
0,2012-08-13 22:54:53.510Z,medium.com,https://medium.com/policy/medium-terms-of-serv...
1,2012-08-13 22:57:17.248Z,medium.com,https://medium.com/policy/medium-privacy-polic...
2,2016-11-04 23:40:43.364Z,medium.com,https://medium.com/@Medium/personalize-your-me...
3,2016-12-24 18:21:13.427Z,medium.com,https://medium.com/holiday-poems/xmas-morning-...
4,2015-09-22 21:37:48.207Z,blog.medium.com,https://blog.medium.com/taking-a-side-on-net-n...
...,...,...,...
95,2016-12-29 09:29:41.795Z,medium.com,https://medium.com/@thisismissem/an-open-cover...
96,2016-12-12 04:54:37.943Z,medium.com,https://medium.com/@kuehn.robin/do-you-have-im...
97,2016-10-13 07:15:36.895Z,hackernoon.com,https://hackernoon.com/asynchronous-server-sid...
98,2015-11-11 11:55:56.456Z,medium.com,https://medium.com/russian/%D0%BA%D0%B0%D0%BA-...
